## Read and parse Log File

Apache access log line example

```
13.66.139.0 - - [19/Dec/2020:13:57:26 +0100] "GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1" 200 32653 "-" "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)" "-"
```

In [0]:
%scala
import java.net.URL
import java.io.File
import org.apache.commons.io.FileUtils
 
val tmpFile = new File("/tmp/access.log")
//FileUtils.copyURLToFile(new URL("https://raw.githubusercontent.com/bdbaraban/holberton-system_engineering-devops/master/0x04-loops_conditions_and_parsing/apache-access.log"), tmpFile)
FileUtils.copyURLToFile(new URL("https://gist.githubusercontent.com/zxcfer/5518d2976a19889e9594ee3923612717/raw/9b1ac12041eda8e23378b01ad0554e93d89e0450/accesslog"), tmpFile)

import java.net.URL
import java.io.File
import org.apache.commons.io.FileUtils
tmpFile: java.io.File = /tmp/access.log

In [0]:
import re
from pyspark.sql import Row

APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+)'

# Parse to dictionary
def parse_apache_log_line(logline):
    try:
        match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
        if match is None:
            # Optionally, you can change this to just ignore if each line of data is not critical.
            # For this example, we want to ensure that the format is consistent.
            raise Exception("Invalid logline: %s" % logline)
        return Row(
            ipAddress    = match.group(1),
            clientIdentd = match.group(2),
            userId       = match.group(3),
            dateTime     = match.group(4),
            method       = match.group(5),
            endpoint     = match.group(6),
            protocol     = match.group(7),
            responseCode = int(match.group(8)),
            trail        = match.group(9)
            )
    except:
        print('err')
        

raw_log_files = sc.textFile("file:/tmp/access.log")
raw_log_files.count()

parsed_log_files = raw_log_files.map(parse_apache_log_line)
df = parsed_log_files.toDF()
df.show(10)

df.createOrReplaceTempView ("log_data")

+---------------+------------+------+--------------------+------+--------------------+--------+------------+------+
|      ipAddress|clientIdentd|userId|            dateTime|method|            endpoint|protocol|responseCode| trail|
+---------------+------------+------+--------------------+------+--------------------+--------+------------+------+
| 195.154.172.53|           -|     -|09/Feb/2016:09:40...|   GET|/wp-content/theme...|HTTP/1.1|         200|   437|
| 195.154.172.59|           -|     -|09/Feb/2016:09:40...|   GET|/wp-content/plugi...|HTTP/1.1|         200|   505|
| 195.154.172.59|           -|     -|09/Feb/2016:09:40...|   GET|/wordpress/wp-con...|HTTP/1.1|         200|  5144|
| 195.154.172.59|           -|     -|09/Feb/2016:09:40...|   GET|/wp-content/theme...|HTTP/1.1|         200|   438|
| 195.154.172.59|           -|     -|09/Feb/2016:09:40...|   GET|/wp-content/uploa...|HTTP/1.1|         200|768386|
|195.154.233.224|           -|     -|09/Feb/2016:09:40...|   GET|/wp-con

## Statistics

In [0]:
%sql
SELECT count(*) as total, responseCode FROM log_data group by responseCode

total,responseCode
45,304
30,404
15,301
530,200
1,400


Output can only be rendered in Databricks